In [50]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [53]:
#为了实现figure弹窗
%matplotlib       

Using matplotlib backend: Qt5Agg


In [50]:
def load_data(filename):
    '''
    读取txt文件
    filename:文件路径+文件名；'./...'
    output:ndarry数组
    '''
    dataset = []
    with open(filename) as f:
        file = f.readlines()#得到raw-list
    for line in file:
        lineArr = line.strip('\n').split('\t') #因为数据类型为'1.658985\t4.285136\n'
        m = len(lineArr)
        dataset.append(lineArr[0:m])
    return np.array(dataset,dtype=np.float64)

>精简版，使用pandas来进行处理

In [54]:
a = pd.read_csv('./testSet.txt',sep='\t',names=['x0','x1'])#pandas操作要比书中源代码简洁快速多了

In [55]:
a.sample(4)

,x0,x1
34,4.372646,-0.822248
62,3.491078,-3.947487
7,-3.487105,-1.724432
41,-2.709034,2.923887


In [59]:
sns.lmplot('x0', 'x1',data=a,fit_reg=False,height=5)
sns.despine(top=False,right=False)

In [57]:
#计算距离
def computeDistance(A,B):
    '''
    计算距离
    '''
    return np.sqrt(np.sum(np.square(A-B)))

In [83]:
np.random.choice(80,4)

array([45,  6, 76, 77])

In [84]:
def randCentroids(x,k=4):
    '''
    初始化质心：从样本中随机选择k个作为质心
    output:质心
    '''
    m,n = x.shape
    centroids = np.zeros((k,n))
    randIndex = np.random.choice(m,k)#随机抽k个值作为索引
    centroids = x[randIndex]
    return centroids

>关于聚类中心初始化，操作就是从ndarray数组中随机取3行，就是对ndarray传入一个len=3的随机list


## 用pandas来实现

想做一个聚类算法演示图

* 根据迭代次数，展示质心和算法的迭代过程，聚类过程
    * 需要解决，如何话gif动图，如何计算每次k-means迭代
    * 每次迭代点的类别，标记出，然后分别用不同颜色画出

>k-means聚类算法的核心：将一组数据分成k个类，通过比较各个数据点和质心的距离，进行分类，并更新每个质心的坐标，直到各个点的类别稳定为止，停止迭代。

* 随机选k个点作为质心
* 计算每个点到k个质心点距离，将其分类到不同的簇中
* 计算每个簇的均值，更新质心
* 迭代，直到质心位置不变为止-->$ \min_{c^{1}\cdots c^{m};\mu^{1}\cdots \mu_{k}} cost  J(c^{1},c^{2}\cdots c^{m},\mu_{1},\mu_{2}\cdots \mu_{k})$

>cost function怎么求？
    1. 计算对应的簇的样本点和质点的2-范数。

>大佬啊，np.apply_along_axis函数

In [16]:
#初始化质心
def random_init(data,k):
    '''
    初始化质心
    参数：
        data:数据集-dataframe
        k:聚类个数
    返回：
        质心数组-matrix
    '''
    return data.sample(k).as_matrix()

#计算样本点距离最小值
def _find_your_cluster(x, centroids):
    """
    将一个样本点归类
    参数:
        x: 一个样本点
        centroids:质心
    返回:
        k: 距离最小值
    """
    distances = np.apply_along_axis(func1d=np.linalg.norm,  # 计算二范数np.linalg.norm
                                    axis=1,
                                    arr=centroids - x)  #利用np广播的特点
    return np.argmin(distances)

#样本点聚类
def assign_cluster(data, centroids):
    """
    将样本点归类
    参数：
        data:数据集
        centroids:质心
    返回：
        类别数组C
    """
    return np.apply_along_axis(lambda x: _find_your_cluster(x, centroids),
                               axis=1,
                               arr=data.as_matrix())#将所有样本点，进行归类

#返回包含聚类后结果列的dataframe
def combine_data_C(data, C):
    data_with_c = data.copy()
    data_with_c['C'] = C
    return data_with_c


#更新质心
def new_centroids(data, C):
    '''
    更新质心
    参数:
        data:数据集
        C:聚类后的类别列
    返回:
        计算每个簇的质心均值
    '''
    data_with_c = combine_data_C(data, C)
    return data_with_c.groupby('C', as_index=False).\
                       mean().\
                       drop('C', axis=1).\
                       as_matrix()#将df按C分组计算行均值，之后删除C列，转化为矩阵

#计算代价函数
def cost(data, centroids, C):
    '''
    计算代价函数：
        让所有数据点同它们对应的类别距离之和最小
    参数：
        data:数据集
        centroids:质心
        C:聚类类别
    返回：
        距离数值
    '''
    m = data.shape[0]

    expand_C_with_centroids = centroids[C]#按聚类结果C扩展质心；为了和data相互匹配x^(i)-u^(i)，u^(i)质心

    distances = np.apply_along_axis(func1d=np.linalg.norm,
                                    axis=1,
                                    arr=data.as_matrix() - expand_C_with_centroids)
    return distances.sum() / m


#k-means聚类算法
def _k_means_iter(data, k, epoch=100, tol=0.0001):
    """
    k-means迭代
    one shot k-means
    with early break
    """
    centroids = random_init(data, k)#初始化随机质点
    cost_progress = []#代价函数list

    #收敛到最小值
    for i in range(epoch):
        print('running epoch {}'.format(i))

        C = assign_cluster(data, centroids)#样本点归类
        centroids = new_centroids(data, C)#更新质心
        cost_progress.append(cost(data, centroids, C))#每次质心的cost

        if len(cost_progress) > 1:  # early break
            if (np.abs(cost_progress[-1] - cost_progress[-2])) / cost_progress[-1] < tol:
                break

    return C, centroids, cost_progress[-1]


#多次随机初始化
def k_means(data, k, epoch=100, n_init=10):
    """
    多次随机初始化取最好的结果返回
    参数:
        data (pd.DataFrame)
    返回:
        (C, centroids, least_cost)
    """

    tries = np.array([_k_means_iter(data, k, epoch) for _ in range(n_init)])

    least_cost_idx = np.argmin(tries[:, -1])#最后一列，是cost值，返回其中最小值

    return tries[least_cost_idx]

### 第一次迭代

In [5]:
a.head()

,x0,x1
0,1.658985,4.285136
1,-3.453687,3.424321
2,4.838138,-1.151539
3,-5.379713,-3.362104
4,0.972564,2.924086


In [6]:
centroids = a.sample(4).as_matrix()         #初始化质心

In [7]:
centroids

array([[ 2.329546,  3.179764],
       [ 2.668759,  1.594842],
       [-0.39237 , -3.963704],
       [ 2.280615, -2.559444]])

In [8]:
C = assign_cluster(a,centroids)   #每个样本归类

In [9]:
dt = combine_data_C(a,C)    #归类后的数据集

In [10]:
new_centroids(a,C)      #新质心

array([[-0.17746937,  3.16790709],
       [ 3.0475435 ,  1.56481175],
       [-2.99723017, -2.84727778],
       [ 3.03713839, -2.62802833]])

In [11]:
cost(a,centroids,C)

2.5915342806611252

In [15]:
_k_means_iter(a,4)  #一次随机质心

running epoch 0
running epoch 1
running epoch 2
running epoch 3
running epoch 4


(array([3, 3, 2, 0, 3, 3, 1, 0, 3, 3, 2, 0, 3, 3, 2, 1, 3, 3, 2, 0, 3, 3,
        2, 0, 3, 3, 2, 0, 3, 3, 1, 0, 3, 3, 2, 0, 3, 3, 2, 0, 3, 3, 2, 1,
        3, 3, 2, 0, 3, 3, 2, 0, 3, 3, 2, 0, 3, 3, 2, 0, 3, 3, 2, 0, 3, 3,
        2, 0, 3, 3, 2, 0, 2, 3, 2, 0, 3, 3, 2, 0]),
 array([[-3.61853111, -2.81946867],
        [-0.28093075, -3.880518  ],
        [ 3.09814284, -2.43041226],
        [-0.02298687,  2.99472915]]),
 1.9025429094043182)

### 多次质心迭代

In [21]:
k_means(a,4)        #多次随机质心

running epoch 0
running epoch 1
running epoch 2
running epoch 3
running epoch 4
running epoch 5
running epoch 6
running epoch 7
running epoch 8
running epoch 9
running epoch 0
running epoch 1
running epoch 2
running epoch 3
running epoch 4
running epoch 5
running epoch 6
running epoch 0
running epoch 1
running epoch 2
running epoch 3
running epoch 0
running epoch 1
running epoch 2
running epoch 3
running epoch 0
running epoch 1
running epoch 2
running epoch 3
running epoch 4
running epoch 5
running epoch 6
running epoch 0
running epoch 1
running epoch 2
running epoch 3
running epoch 0
running epoch 1
running epoch 2
running epoch 3
running epoch 4
running epoch 5
running epoch 6
running epoch 7
running epoch 8
running epoch 9
running epoch 0
running epoch 1
running epoch 2
running epoch 3
running epoch 0
running epoch 1
running epoch 2
running epoch 3
running epoch 4
running epoch 5
running epoch 6
running epoch 0
running epoch 1
running epoch 2
running epoch 3
running epoch 4
running 

array([array([3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 1, 3, 0, 1, 2, 3, 0,
       1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2,
       3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0,
       1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2]),
       array([[-2.46154315,  2.78737555],
       [ 2.65077367, -2.79019029],
       [-3.53973889, -2.89384326],
       [ 2.6265299 ,  3.10868015]]),
       1.1675654672086733], dtype=object)

In [22]:
result = k_means(a,4)  

running epoch 0
running epoch 1
running epoch 2
running epoch 3
running epoch 0
running epoch 1
running epoch 2
running epoch 3
running epoch 4
running epoch 5
running epoch 6
running epoch 0
running epoch 1
running epoch 2
running epoch 3
running epoch 0
running epoch 1
running epoch 2
running epoch 3
running epoch 0
running epoch 1
running epoch 2
running epoch 3
running epoch 4
running epoch 0
running epoch 1
running epoch 2
running epoch 3
running epoch 4
running epoch 5
running epoch 6
running epoch 0
running epoch 1
running epoch 2
running epoch 3
running epoch 0
running epoch 1
running epoch 2
running epoch 3
running epoch 0
running epoch 1
running epoch 2
running epoch 3
running epoch 4
running epoch 5
running epoch 0
running epoch 1
running epoch 2
running epoch 3
running epoch 4
running epoch 5
running epoch 6
running epoch 7
running epoch 8
running epoch 9
running epoch 10
running epoch 11


###  每次迭代保存结果
获得每次迭代的结果，来可视化迭代的过程

In [35]:
#k-means聚类算法
def _k_means_iter_each(data, k, epoch=100, tol=0.0001):
    """
    保存每个k-means迭代的分类结果
    参数：
        data:数据集
        k:聚类簇数量
    返回：
        每个分类C，质心，以及cost
    """
    centroids = random_init(data, k)#初始化随机质点
    cost_progress = []#代价函数list
    
    '''
    #添加list，来保存每次迭代的结果
    '''
    each_C = []
    each_centroids = []
    
    #收敛到最小值
    for i in range(epoch):
        print('running epoch {}'.format(i))

        C = assign_cluster(data, centroids)#样本点归类
        centroids = new_centroids(data, C)#更新质心
        cost_progress.append(cost(data, centroids, C))#每次质心的cost
        each_C.append(C)
        each_centroids.append(centroids)

        if len(cost_progress) > 1:  # early break
            if (np.abs(cost_progress[-1] - cost_progress[-2])) / cost_progress[-1] < tol:
                break

    return each_C, each_centroids, cost_progress

In [36]:
def get_data(a,k):
    '''
    得到每次迭代的结果
    参数：
        a:原数据集
        k:质心数量
    返回：
        迭代结果的list
    '''
    result = _k_means_iter_each(a,k)#每次迭代的数据结果
    C = result[0]
    mid = []
    for i in C:
        b = a.copy()
        dt = combine_data_C(b,i)
        mid.append(dt)
    return mid

In [44]:
def draw_gif():
    '''
    画gif图
    将每次迭代的结果保存为png，再调用imageio生成Gif。
    '''
    import matplotlib.pyplot as plt
    import imageio,os
    re = get_data(a,4)
    co = sns.color_palette("RdBu",4)
    #保存每个epoch结果
    for i in range(len(re)):
        sns.relplot('x0','x1',data=re[i],hue='C',palette=co);
        plt.savefig("fig"+str(i)+".png",dpi = 100)
        
    #做gif图
    images = []
    filenames=sorted((fn for fn in os.listdir('.') if fn.endswith('.png')))
    for filename in filenames:
        images.append(imageio.imread(filename))
    imageio.mimsave('gif.gif', images,duration=1)

In [45]:
for i in range(10):
    draw_gif()

running epoch 0
running epoch 1
running epoch 2
running epoch 3
running epoch 4
running epoch 0
running epoch 1
running epoch 2
running epoch 3
running epoch 4
running epoch 5
running epoch 6
running epoch 0
running epoch 1
running epoch 2
running epoch 3
running epoch 4
running epoch 5
running epoch 6
running epoch 0
running epoch 1
running epoch 2
running epoch 3
running epoch 4
running epoch 5
running epoch 6
running epoch 7
running epoch 8
running epoch 9
running epoch 0
running epoch 1
running epoch 2
running epoch 3
running epoch 0
running epoch 1
running epoch 2
running epoch 3
running epoch 0
running epoch 1
running epoch 2
running epoch 0
running epoch 1
running epoch 2
running epoch 3
running epoch 4
running epoch 5
running epoch 6
running epoch 7
running epoch 8
running epoch 0
running epoch 1
running epoch 2
running epoch 3
running epoch 0
running epoch 1
running epoch 2
running epoch 3
running epoch 4
